# Setup

In [5]:
import numpy as np
#%matplotlib widget
from matplotlib import pyplot as plt
import os, time
from tpo_tools.analysis import CollectionTemplateWF, PUNDTemplateWF, SampleInfo
from tpo_tools.instrumentation import AWGSettings, AWGChannelSettings, OscilloscopeChannelSettings, OscilloscopeSettings, RigolDHO1074, RigolDG2102
from tpo_tools.analysis import SineTemplateWF, ConstantTemplateWF, TrapezoidTemplateWF, DUTInfo
from tpo_tools.analysis import TemplatedTransportData, TransportDataList, PUND
from tpo_tools import fig2clipboard, figures2gif
outdir = './2025-08-01_big_cap/'

try:
    os.mkdir(outdir)
except:
    pass

# Set up AWG/Oscope

In [7]:
#Connect to devices
oscope = RigolDHO1074.identify('TCPIP0::10.97.108.250::INSTR')
oscope.verbose= False
oscope.delay = 0.01
awg = RigolDG2102.identify('TCPIP0::10.97.108.216::INSTR')
awg.verbose= False
awg.delay = 0.25

awg.reset()
oscope.reset()
state_set_counter = 0

At address TCPIP0::10.97.108.250::INSTR, RigolDHO1074 query result: RIGOL TECHNOLOGIES,DHO1074,HDO1B244902222,00.02.13
At address TCPIP0::10.97.108.216::INSTR, RigolDG2102 query result: Rigol Technologies,DG2102,DG2P233800951,00.02.07.00.01


AWG and oscilloscope settings. 

In [ ]:
sample_info = SampleInfo(id='Big cap', notes='Redoing NLS')
DUTInfo(name='FeCap', area=300**2/4*np.pi, thickness=20e-9, parent=sample_info) # 9 um^2, 10 nm thick
DUTInfo(name='Heater', area=1e-14, thickness=0, parent=sample_info)


def reset(ch1=True, ch2=True):
    awg_settings = AWGSettings()
    oscope_settings = OscilloscopeSettings(sample_rate=2e6, trig_delay=1e-4)

    if ch1:
        awgch1 = AWGChannelSettings(channel=1, sample_rate=1e5, destination='FeCap',  parent=awg_settings)
        OscilloscopeChannelSettings(channel=1, vrange=10,                      source='FeCap',  parent=oscope_settings)
        OscilloscopeChannelSettings(channel=2, vrange=10, transimpedance=-1e3, source='FeCap',  parent=oscope_settings)
        # amplifier is a inverted  current amp, -1mA goes to 1V
        CollectionTemplateWF(parent=awgch1, destination='FeCap')
    else:
        raise ValueError('Always watching the FeCap...')
    
    # ch2 is for the heater
    if ch2:
        awgch2 = AWGChannelSettings(channel=2, sample_rate=1e5, destination='Heater', parent=awg_settings)
        OscilloscopeChannelSettings(channel=3, vrange=5,                       source='Heater', parent=oscope_settings)
        OscilloscopeChannelSettings(channel=4, vrange=5,  transimpedance=+1e4, source='Heater', parent=oscope_settings)
        CollectionTemplateWF(parent=awgch2, destination='Heater')
    else:
        awgch2 = AWGChannelSettings(channel=2, sample_rate=1e5, destination='none', parent=awg_settings)
        OscilloscopeChannelSettings(channel=3, source='none', parent=oscope_settings)
        OscilloscopeChannelSettings(channel=4, source='none', parent=oscope_settings)
        CollectionTemplateWF(parent=awgch2, destination='none')
    
    return awg_settings, oscope_settings


In [9]:
def set_state(awg_settings, oscope_settings):
    for _ in range(2): # need to run through this twice after first turning on AWG
        # Update awg settings first
        try:
            assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'
        except:
            try:
                assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'
            except:
                assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'

        # Then update oscope settings
        try:
            assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'
        except:
            try:
                assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'
            except:
                assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'

        if oscope_settings.sample_rate != 2e6:
            print('Oscope sample rate adjusted to be', oscope_settings.sample_rate)

        time.sleep(0.2)

# Custom waveforms

In [ ]:
cycle_voltage = 4

def set_cycle(NDPU=False):
    awg_settings, oscope_settings = reset(ch2=False)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template
    oscope_settings.sample_rate = 5e5

    polarity = -1 if NDPU else +1 

    PUNDTemplateWF(amplitude=cycle_voltage*polarity, dVdt=1e4, delay_time=0, n_cycles=1e3, n_pulses=1, parent=template1)
    # if n_pulses = 1, it goes like PNPNPNPN---

    set_state(awg_settings, oscope_settings)
    global state
    state = 'cycle-ndpu' if NDPU else 'cycle-pund'
    print('~~~~~~~~~~ Set state for', state, '~~~~~~~~~~~~')

def set_PUND(NDPU=False):
    awg_settings, oscope_settings = reset(ch2=False)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template

    polarity = -1 if NDPU else +1

    PUNDTemplateWF(amplitude=cycle_voltage*polarity, dVdt=1e4, delay_time=5e-3, n_cycles=4, parent=template1)

    set_state(awg_settings, oscope_settings)
    global state
    state = 'ndpu' if NDPU else 'pund'
    print('~~~~~~~~~~ Set state for', state, '~~~~~~~~~~~~')

def set_cycle_wait_PUND(wait_time_exp,NDPU=False):
    awg_settings, oscope_settings = reset(ch2=False)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template

    polarity = -1 if NDPU else +1 
    wait_time = 10**wait_time_exp # in seconds
    print('Wait time: '+str(wait_time))
    PUNDTemplateWF(amplitude=cycle_voltage*polarity, dVdt=1e4, delay_time=0, n_cycles=1e4, n_pulses=1, parent=template1)
    ConstantTemplateWF(value=0, duration=wait_time, parent=template1)
    PUNDTemplateWF(amplitude=cycle_voltage*polarity, dVdt=1e4, delay_time=5e-3, n_cycles=4, parent=template1)
    oscope_settings.trig_delay = -1 * (wait_time + cycle_voltage * 4 - 1e-04) #cycle_voltage / dVdt * 2 = time for one pulse, * 2 for the negative pulse * n_cycles

    set_state(awg_settings, oscope_settings)
    global state
    state = f'NP_1e{wait_time_exp:.2f}_NDPU' if NDPU else f'PN_1e{wait_time_exp:.2f}_PUND'
    print('~~~~~~~~~~ Set state for', state, '~~~~~~~~~~~~')


state = ''
#set_cycle_wait_PUND(-4, NDPU=False)
print(state)

In [6]:
def trigger(save=True):
    oscope.prepare_for_trigger()
    awg.prepare_for_trigger()
    oscope.trigger()

    time.sleep(oscope.awg_settings.duration) # wait for wf to finish
    data = oscope.read_data()

    awg.done_with_trigger() # turn off outputs
    oscope.done_with_trigger() 

    data.sample_info = sample_info 
    print('Data read') 

    if save:
        data.save(f'{outdir}/{data[0].daq_timeofday}_{state}.hdf5') 
    return data
#trigger(save=False)

## Take/Save Data

In [7]:
pund_data = []

import time

# Main loop to collect data

set_PUND(NDPU=False)
pund_data.append( trigger() )

for t in np.arange(-4,2,1/3):
    set_cycle_wait_PUND(t,NDPU=False)
    pund_data.append( trigger() ) 

    set_cycle_wait_PUND(t,NDPU=True)
    pund_data.append( trigger() ) 

for t in np.arange(2,4.4,1/3):
    print('wait_exp =', t)
    set_cycle(NDPU=False)
    pund_data.append( trigger() )
    start = time.time()
    set_PUND(NDPU=False)
    stop = time.time()
    setup_time = stop - start
    time.sleep(10**t-setup_time)
    pund_data.append( trigger() )

    print('wait_exp =', t)
    set_cycle(NDPU=True)
    pund_data.append( trigger() )
    start = time.time()
    set_PUND(NDPU=True)
    stop = time.time()
    setup_time = stop - start
    time.sleep(10**t-setup_time)
    pund_data.append( trigger() )

print('Done!')

Offset	 0.0499 	 4.990000E-2
Memdep	 1000000.0 	 1.000E+06
Secdiv	 0.01 	 1.000000E-2
Offset	 0.0499 	 4.990000E-2
Memdep	 1000000.0 	 1.000E+06
Secdiv	 0.01 	 1.000000E-2
~~~~~~~~~~ Set state for pund ~~~~~~~~~~~~
Data read
Wait time: 0.0001
Offset	 13.0 	 1.300000E+1
Memdep	 10000000.0 	 1.000E+07
Secdiv	 0.2 	 2.000000E-1
Offset	 13.0 	 1.300000E+1
Memdep	 10000000.0 	 1.000E+07
Secdiv	 0.2 	 2.000000E-1
~~~~~~~~~~ Set state for PN_1e-4.00_PUND ~~~~~~~~~~~~
Data read
Wait time: 0.0001
Offset	 13.0 	 1.300000E+1
Memdep	 10000000.0 	 1.000E+07
Secdiv	 0.2 	 2.000000E-1
Offset	 13.0 	 1.300000E+1
Memdep	 10000000.0 	 1.000E+07
Secdiv	 0.2 	 2.000000E-1
~~~~~~~~~~ Set state for NP_1e-4.00_NDPU ~~~~~~~~~~~~
Data read
Wait time: 0.00021544346900318845
Offset	 13.000115443469003 	 1.300012E+1
Memdep	 10000000.0 	 1.000E+07
Secdiv	 0.2 	 2.000000E-1
Offset	 13.000115443469003 	 1.300012E+1
Memdep	 10000000.0 	 1.000E+07
Secdiv	 0.2 	 2.000000E-1
~~~~~~~~~~ Set state for PN_1e-3.67_PUND ~~~~